## Домашнее задание 6

В данном домашнем задании Вам предстоит реализовать автоматическое исправление опечаток в запросах пользователей. 

### 1. Датасет
Для оценки качества алгоритма исправления опечаток, Вам предоставляется файл `queries.tsv.gz`. В каждой строке файла записаны два запроса – исходный и исправленный. Для простоты, оба запроса будут иметь одинаковое количество слов и отличаться незначительно. Зачастую исходный и исправленный запрос совпадают, что означает что исправлять такой запрос не требуется.

In [2]:
from typing import List, Tuple, Generator, Callable

Query = str
Sentence = str
Filename = str
Word = str
Queries = List[Tuple[Query, Query]]

In [3]:
from termcolor import colored
import difflib

def diff_queries(original: Query, fixed: Query) -> Query:
    result = ''
    for pos, d in enumerate(difflib.ndiff(original, fixed)):
        if d[0] == '+':
            result += colored(d[2], 'green')
        elif d[0] == '-':
            result += colored(d[2], 'red')
        else:
            result += d[2]
    return result

print(diff_queries("lake compond the park", "lake compound the park"))
print(diff_queries("traditional chothes", "traditional clothes"))
print(diff_queries("jack sparrow", "captain jack sparrow"))

lake compound the park
traditional chlothes
captain jack sparrow


In [13]:
import gzip

def load_queries(fn: Filename) -> Queries:
    result = []
    with gzip.open(fn, 'rt', encoding='utf8') as inp:
        for line in inp:
            original, fixed = line.rstrip('\n').split('\t')
            result.append((original, fixed))
    return result

queries = load_queries("queries.tsv.gz")
print(f'Loaded {len(queries)} queries\n')
for original, fixed in queries[10:20]:
    print(diff_queries(original, fixed))

Loaded 102436 queries

embearirassing red carpet moments
grants for rural areas florirda
the home hdepot merchandising
delaware motorcycle inspection requirements
highland park hospital gastric biypass surgery
grand theift auto
windward community college
my credit reports
strack intermediate school
mongol empire political system


In [5]:
queries_sample = [
    ("grand theift auto", "grand theft auto"),
    ("belarus longitude and latitdue", "belarus longitude and latitude"),
    ("search for poeoms", "search for poems"),
    ("large guacolmoi dip restaurtant price", "large guacamole dip restaurant price"),
    ("texas chainsaw mascurer", "texas chainsaw massacre"),
    ("royal trump subtitle", "royal tramp subtitle"),
    ("florida fiberglass polls", "florida fiberglass pools"),
    ("how to make a calender", "how to make a calendar"),
    ("university of south caroline", "university of south carolina"),
    ("maureen mcdonald in virginia", "maureen mcdonnell in virginia"),
]

Для составления словаря и обучения языковых моделей Вам предоставляется небольшой корпус текста, неслучайная выборка из большой английской википедии в файле `train.bz2`. Этот файл содержит примерно 5 млн строк или 80 млн слов. Каждая строка – одно предложение без знаков препинания.
Использование других словарей и корпусов запрещено.

In [7]:
import bz2
from tqdm import tqdm

def read_huge_corpus(fn: Filename) -> Generator[Sentence, None, None]:
    with bz2.open(fn, 'rt', encoding='utf8') as inp:
        for line in tqdm(inp):
            yield line.rstrip('\n')

for li, line in enumerate(read_huge_corpus("./train.bz2")):
    print(line)
    if li == 10:
        break

10it [00:00, 237.45it/s]

gol neshin
mitochondrial dna depletion syndrome mds or mdds is any of a group of autosomal recessive disorders that cause a significant drop in mitochondrial dna in affected tissues
following the relegation of sc freiburg in 2005 he was on the verge of signing for metalurg donetsk but instead he accepted a contract with vfl wolfsburg
the first issue for geometers is what kind of geometry is adequate for a novel situation
cedar grove was formerly a stage and freight stop
regular bus service runs from bhubaneswar to niali which is away
later they were also known for the cream wafer biscuits
strabomantis cornutus
gtk+ scene graph kit gsk was initially released as part of gtk+ 3.90 in march 2017 and is meant for gtk-based applications that wish to replace clutter for their ui
the match took place on 10 april 1906 at the hipódromo madrid
the brothers came from fresno california


### 2. Поиск близких слов
Требуется научится быстро находить список из сотни слов, которые незначительно отличаются от заданного слова.

Не стоит перебирать все слова словаря – займёт слишком много времени.

Для ускорения перебора предлагается создать триграммный индекс – для каждой буквенной триграммы храним список слов, в которых она есть. Тогда для поиска похожих на данное слово найдем слова большим количеством совпадающих триграмм. 

Совет 1: стоит сделать отельный индекс для каждой длинны слова и использовать только те индексы, в которых лежат слова близкие по длине к исходному.

Совет 2: для выделения триграмм стоит обрамить слово спецсимволом, чтобы триграммы на концах слова отличались от оных в середине.

Любые другие алгоритмы, улучшающие качество за разумное время (хождение по бору с ошибками, перебор ошибок) – не возбраняются.

Не побрезгуйте кешировать результат работы этого алгоритма, чтобы дальнейшая работа протекала быстрее.

In [8]:
import pickle
from nltk.tokenize import word_tokenize
import os.path

In [9]:
from collections import defaultdict

if os.path.exists("all_words"):
    all_words = pickle.load(open("all_words", 'rb'))
else:
    all_words = defaultdict(int)
    train = read_huge_corpus("./train.bz2")
    for ind, text in enumerate(train):
        for word in text.split(" "):
            all_words[word] += 1
    all_words = {i: all_words[i] for i in all_words if all_words[i] > 5}
    pickle.dump(all_words, open("all_words", 'wb'))

In [10]:
trigram_index = defaultdict(lambda: defaultdict(set))

def trigrams(word, m=3):
    padded = "$$" + word + "$$"
    return [padded[i:i+m] for i in range(len(padded)-m+1)]

for word in tqdm(all_words.keys()):
    for trigram in trigrams(word):
        trigram_index[len(word)][trigram].add(word)

100%|██████████| 228997/228997 [00:02<00:00, 79931.41it/s]


In [16]:
words_to_fix = []#[(o, f) for o, f in zip(query[0].split(" "), query[1].split(" ")) if o != f]
for query in tqdm(queries):
    words_to_fix += [(o, f) for o, f in zip(query[0].split(" "), query[1].split(" ")) if o != f]
words_to_fix[:10]

100%|██████████| 102436/102436 [00:00<00:00, 361629.85it/s]


[('chothes', 'clothes'),
 ('cataloges', 'catalogs'),
 ('compond', 'compound'),
 ('barns', 'barnes'),
 ('emberissing', 'embarrassing'),
 ('floirda', 'florida'),
 ('hepot', 'depot'),
 ('inspectio', 'inspection'),
 ('bipass', 'bypass'),
 ('theift', 'theft')]

In [17]:
from collections import Counter
from functools import lru_cache 

@lru_cache(maxsize=None)
def find_similar_words(word: Word, len_gap=2, N=1000) -> List[Word]:
    similar = []
    for trigram in trigrams(word):
        for word_len in range(len(word) - len_gap, len(word) + len_gap + 1):
            if trigram in trigram_index[word_len]:
                similar += trigram_index[word_len][trigram]
    similar = Counter(similar)
    # TODO: replace with partial sort
    return sorted(
        list(similar.keys()), 
        key=lambda x:(similar[x] / len(word), -abs(len(x) - len(word))),
        reverse=True)[:N]

for original, fixed in words_to_fix[:5]:
    similar = find_similar_words(original)
    print(original, '- ok' if fixed in similar else '- fail')
    for word in similar[:5]:
        print(' ', word)
    print()

chothes - ok
  clothes
  choices
  crathes
  chooses
  chores

cataloges - ok
  catalogues
  cataloged
  catalogus
  catalogs
  catalyses

compond - ok
  compound
  composed
  component
  commend
  compose

barns - ok
  barns
  bairns
  barnes
  barnas
  barons

emberissing - ok
  embarrassing
  embossing
  remembering
  embezzling
  embellishing



Чтобы оценить качество полученного алгоритма, используйте запросы из `queries.tsv.gz`. Отберите только отличающиеся слова в исправленном и исходном запросах. Проверьте, что для слова в исходном запросе, исправленное слово будет в списке ближайших выданном вашим алгоритмом. Если это выполняется для всех или почти всех пар – успех. 

In [18]:
def extract_different_words(queries: Queries) -> List[Tuple[Word, Word]]:
    words_to_fix = []
    for original, fixed in queries:
        if original != fixed:
            for word_orig, word_fixed in zip(original.split(), fixed.split()):
                if word_orig != word_fixed:
                    words_to_fix.append((word_orig, word_fixed))
    return words_to_fix
                    
words_to_fix = extract_different_words(queries)
print(f'Found {len(words_to_fix)} words to fix')
for original, fixed in words_to_fix[:10]:
    print(diff_queries(original, fixed))

Found 53495 words to fix
chlothes
cataloges
compound
barnes
embearirassing
florirda
hdepot
inspection
biypass
theift


In [156]:
word_to_similar = {}

def check_find_similar_words(words_to_fix: List[Tuple[Word, Word]], 
                             find_similar_words: Callable[[Word], List[Word]]):
    wrong, total = 0, 0
    progress = tqdm(words_to_fix)
    for word_orig, word_fixed in progress:
        similar = find_similar_words(word_orig)
        if word_fixed not in similar:
            wrong += 1
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_find_similar_words(words_to_fix, find_similar_words)

Wrong: 115 - 6.86%:   3%|▎         | 1675/53495 [01:18<40:27, 21.35it/s] 


KeyboardInterrupt: 

## 3. Языковая модель
Языковая модель – модель, которая по тексту оценивает вероятность того, что он мог появиться в языке. 

Постройте простую n-грамную языковую модель с использованием корпуса текстов `train.bz2`. Для этого рассчитайте количество вхождений каждой n-граммы в корпус текста. Если взять n=2, то размера оперативной памяти вашего компьютера должно будет хватить.

Воспользуйтесь каким-нибудь методом сглаживания, чтобы не получать нулевую вероятность для неизвестных n-грамм. Также, чтобы вероятности слов, которых нет в словаре, были отличны от нуля, можно примешать побуквенную m-граммную модель.

Совет N: если количество оперативной памяти прижмёт, можно хранить строки в виде байт – один раскодированный символ занимает больше памяти чем один байт, при этом для английского текста почти всегда один символ кодируется одним байтом.

In [19]:
n = 2
m = 3

def word_ngram(words, n):
    return [" ".join(words[i:i+n]) for i in range(len(words)-n+1)]

if os.path.exists("n_grams"):
    n_grams = pickle.load(open("n_grams", 'rb'))
    m_grams = pickle.load(open("m_grams", 'rb'))
else:
    n_grams = defaultdict(int)
    m_grams = defaultdict(int)
    train = read_huge_corpus("./train.bz2")
    for text in train:
        words = text.split(" ")
        for n_gram in word_ngram(words, n):
            n_grams[n_gram] += 1
            
        for word in text.split(" "):
            for m_gram in trigrams(word, m):
                m_grams[m_gram] += 1

    pickle.dump(n_grams, open("n_grams", 'wb'))
    pickle.dump(m_grams, open("m_grams", 'wb'))

total_n_grams = sum(n_grams[n_gram] for n_gram in n_grams)
total_m_grams = sum(m_grams[m_gram] for m_gram in m_grams)

In [20]:
from math import log2

total_words = sum(all_words[word] for word in all_words)

def get_probability(query: Query) -> float:
    probability = 1
    words = query.split(" ")
    for n_gram in word_ngram(words, n):
        if n_gram not in n_grams:
            probability *= 1 - n / len(n_grams)
            for word in n_gram.split(" "):
                if word not in all_words:
                    probability *= 1 - 1 / len(words)
                    for m_gram in trigrams(word, m):
                        if m_gram not in m_grams:
                            probability = 1 - all_words[word] / len(m_grams)
    return probability

for original, fixed in queries_sample:
    p_original = get_probability(original)
    p_fixed = get_probability(fixed)
    verdict = '[ok]  ' if p_fixed > p_original else '[fail]'
    sign = '< ' if p_fixed > p_original else '>='
    print(f'{verdict} {original:>40s} {p_original:5.2f}  {sign} {p_fixed:5.2f} {fixed}')

[ok]                          grand theift auto  0.44  <   1.00 grand theft auto
[ok]             belarus longitude and latitdue  0.75  <   1.00 belarus longitude and latitude
[ok]                          search for poeoms  0.67  <   1.00 search for poems
[ok]      large guacolmoi dip restaurtant price  0.41  <   0.80 large guacamole dip restaurant price
[ok]                    texas chainsaw mascurer  0.67  <   1.00 texas chainsaw massacre
[ok]                       royal trump subtitle  1.00  <   1.00 royal tramp subtitle
[fail]                 florida fiberglass polls  1.00  >=  1.00 florida fiberglass pools
[ok]                     how to make a calender  0.80  <   1.00 how to make a calendar
[ok]               university of south caroline  1.00  <   1.00 university of south carolina
[fail]             maureen mcdonald in virginia  1.00  >=  1.00 maureen mcdonnell in virginia


Чтобы оценить качество полученной модели, используйте запросы из `queries.tsv.gz`. Сравните вероятность, которую выдает ваша модель для исходных и исправленных запросов. Хорошая модель выдаёт исправленному запросу большую вероятность. 

In [174]:
def check_language_model(queries: Queries, get_probability: Callable[[Query], float], debug: bool):
    wrong, total = 0, 0
    progress = tqdm(queries)
    debug_output = 0
    for original, fixed in progress:
        if original == fixed:
            continue
        p_original = get_probability(original)
        p_fixed = get_probability(fixed)
        if p_fixed <= p_original:
            wrong += 1
            if debug:
                print(original, p_original)
                print(fixed, p_fixed)
                print()
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_language_model(queries, get_probability, debug=False)

Wrong: 141 - 8.38%:   3%|▎         | 3346/102436 [00:06<03:06, 531.55it/s]


KeyError: 'miami3'

Советую сохранить полученную модель на диск – а случае чего, чтение статистик с диска, может быть быстрее расчёта оных с нуля.

### 4. Модель ошибок
Модель ошибок – модель которая по исходному и исправленному запросу оценивает вероятность того, что такая ошибка могла быть допущена.

Рассчитайте простую модель ошибок на основе расстояния Дамерау-Левенштейна, то есть модифицированного Левенштейна, который считает перестановку соседних букв за одну ошибку.

In [21]:
from math import log2
from fastDamerauLevenshtein import damerauLevenshtein

def get_error_probability(original: Query, fixed: Query) -> float:
    return damerauLevenshtein(original, fixed, similarity=True)

for original, fixed in queries_sample:
    p_error = get_error_probability(original, fixed)
    print(f'{original:>40s} | {p_error:5.2f} | {fixed}')

                       grand theift auto |  0.94 | grand theft auto
          belarus longitude and latitdue |  0.97 | belarus longitude and latitude
                       search for poeoms |  0.94 | search for poems
   large guacolmoi dip restaurtant price |  0.86 | large guacamole dip restaurant price
                 texas chainsaw mascurer |  0.83 | texas chainsaw massacre
                    royal trump subtitle |  0.95 | royal tramp subtitle
                florida fiberglass polls |  0.96 | florida fiberglass pools
                  how to make a calender |  0.95 | how to make a calendar
            university of south caroline |  0.96 | university of south carolina
            maureen mcdonald in virginia |  0.90 | maureen mcdonnell in virginia


## 5. Олтугеза
Объедините результат работы предыдущих пунктов в единый алгоритм исправления опечатки для запроса.

Примерный план:
1.	Для слов запроса генерируем список ближайших слов-кандидатов (для всех, даже словарных слов).
2.	Собираем список кандидатов-запросов (эвристически, чтобы не сделать экспоненциальное время выполнения)
3.	Для каждого кандидата считаем итоговый объединенный score на основе языковой модели и модели ошибок для данного кандидата (не обязательно сумма или произведение, можно объединение любой сложности).
4.	Выдаём гипотезу с наибольшим score.
5.	???
6.	Profit

In [22]:
from random import choice
from itertools import product

def correct(query: Query, words_top=2) -> Query:
    queries = []
    similar = {}
    for word in query.split(" "):
        similar[word] = sorted(
            find_similar_words(word),
            key=lambda x: -get_error_probability(word, x)
        )[:words_top]
    similar_queries = sorted(
        [" ".join(pr) for pr in product(*[similar[word] for word in query.split(" ")])],
        key=lambda x: get_probability(x),
        reverse=True
    )
    return similar_queries[0]

for original, fixed in queries_sample:
    predict = correct(original)
    verdict = '[ok]  ' if predict == fixed else '[fail]'
    sign = '==' if predict == fixed else '!='
    print(f'{verdict} {predict:>40s} {sign} {fixed}')

[ok]                           grand theft auto == grand theft auto
[ok]             belarus longitude and latitude == belarus longitude and latitude
[ok]                           search for poems == search for poems
[fail]       large giacomo dip restaurant price != large guacamole dip restaurant price
[fail]                    texas chainsaw maurer != texas chainsaw massacre
[fail]                     royal trump subtitle != royal tramp subtitle
[fail]                 florida fiberglass polls != florida fiberglass pools
[ok]                     how to make a calendar == how to make a calendar
[fail]             university of south caroline != university of south carolina
[fail]             maureen mcdonald in virginia != maureen mcdonnell in virginia


Итоговое качество меряем на примерах из `queries.tsv.gz`.

Для отладки проблем с качеством имеет смысл научится понимать на каком этапе теряется правильная гипотеза для каждого примера. Например, если правильное исправление есть в списке кандидатов (п. 2), но не выбирается как лучшая – стоит крутить языковую модель, модель ошибок и их объединение.

In [23]:
def check_corrector(queries: Queries, correct: Callable[[Query], Query], debug: bool):
    wrong, total = 0, 0
    progress = tqdm(queries)
    debug_output = 0
    for original, fixed in progress:
        if len(original.split(' ')) > 7:
            continue
        predict = correct(original)
        if predict != fixed:
            wrong += 1
            if debug:
                print(original)
                print(fixed)
                print(predict)
                print()
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_corrector(queries, correct, debug=False)

Wrong: 2722 - 25.25%:  11%|█         | 11162/102436 [10:12<1:23:30, 18.22it/s]


KeyboardInterrupt: 